# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4928, uptime 0:00:34
mini_beamline                    RUNNING   pid 4929, uptime 0:00:34
random_walk                      RUNNING   pid 4930, uptime 0:00:34
random_walk_horiz                RUNNING   pid 4931, uptime 0:00:34
random_walk_vert                 RUNNING   pid 4932, uptime 0:00:34
simple                           RUNNING   pid 4933, uptime 0:00:34
thermo_sim                       RUNNING   pid 4934, uptime 0:00:34
trigger_with_pc                  RUNNING   pid 4935, uptime 0:00:34


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-06-11 20:12:42
Persistent Unique Scan ID: '86f50e82-aa6b-421e-8b0c-0aae28c1ad94'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 20:12:42.7 |     -1.000 |      99615 |
|         2 | 20:12:42.8 |     -0.500 |     101405 |
|         3 | 20:12:42.8 |      0.000 |     101596 |
|         4 | 20:12:42.8 |      0.500 |     101024 |


|         5 | 20:12:42.8 |      1.000 |      98627 |


+-----------+------------+------------+------------+
generator scan ['86f50e82'] (scan num: 3)





('86f50e82-aa6b-421e-8b0c-0aae28c1ad94',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/86f50e82-aa6b-421e-8b0c-0aae28c1ad94-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-06-11 20:12:43
Persistent Unique Scan ID: '64850d91-1af2-423b-bddf-d726d99c5573'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 20:12:43.3 |          -1 | 1085639320 |


|         2 | 20:12:43.4 |          -0 | 1070132548 |


|         3 | 20:12:43.5 |           0 | 1064366481 |


|         4 | 20:12:43.7 |           0 | 1060172256 |


|         5 | 20:12:43.8 |           1 | 1056427867 |


+-----------+------------+-------------+------------+
generator scan ['64850d91'] (scan num: 4)





('64850d91-1af2-423b-bddf-d726d99c5573',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-06-11 20:12:44
Persistent Unique Scan ID: '12ecde69-ade1-47cc-a2d3-2fb6e34fc4ce'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 20:12:44.3 | 1054326748 |


|         2 | 20:12:45.2 | 1105688044 |


|         3 | 20:12:46.2 | 1129409826 |


...

|        28 | 20:13:11.3 | 1076426604 |


|        29 | 20:13:12.3 | 1056864986 |


|        30 | 20:13:13.3 | 1109260578 |


+-----------+------------+------------+
generator count ['12ecde69'] (scan num: 5)





('12ecde69-ade1-47cc-a2d3-2fb6e34fc4ce',)